In [2]:
import csv
import requests
import pandas as pd
import time
import json
import os
import sys
from io import StringIO

In [3]:
API_KEY = "EH1OGFJKB5ESXU34"
POLYGON_API_KEY = "4247kuYEqP7gcbp5dfn7WNIn8DJCEcN_"
DATA_PATH = './data'

# Fetching all Tickers

In [ ]:
Listings_URL = f"https://www.alphavantage.co/query?function=LISTING_STATUS&apikey={API_KEY}"
with requests.Session() as s:
    download = s.get(Listings_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    with open(f'{DATA_PATH}/tickers/active_tickers.csv', 'w+', newline='') as f:
        basic_writer = csv.writer(f)
        basic_writer.writerows(cr)

In [ ]:
raw_df = pd.read_csv(f'{DATA_PATH}/tickers/active_tickers.csv')
raw_df = raw_df.drop(["delistingDate", "status"], axis = 1)
raw_df = raw_df[raw_df['assetType'] == 'Stock']
raw_df = raw_df[(raw_df['exchange'] == 'NYSE') | (raw_df['exchange'] == 'NASDAQ')]
raw_df = raw_df[raw_df['symbol'].str.contains('-') == False]
raw_df = raw_df[raw_df['symbol'].str.len() <= 4]
print(raw_df)

# Fetching Market Capitalization for all stock tickers

In [ ]:
with open(f'{DATA_PATH}/tickers/stock_companies.csv', 'w+', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    
    header = ['Symbol', 'Name', 'CIK', 'Sector', 'Industry', 'MarketCapitalization', 'EBITDA', 'PERatio']
    writer.writerow(header)
    
    unprocessed = []
    i = 0
    for row in raw_df.iterrows():
        
        print(i)
        i = i+1
        
        symbol = row[1]['symbol']
        url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={API_KEY}'
        
        r = requests.get(url)
        
        if r.status_code != 200:
            unprocessed.append(symbol)
            print(f"Incorrect status code for symbol {symbol}. Expected 200, received {r.status_code}")
            continue

        data = r.json()
            
        if not 'Symbol' in data:
            unprocessed.append(symbol)
            print(f"Problem with symbol {symbol}. Received \n{data}")
            continue
        
        info = [data['Symbol'], data['Name'], data['CIK'], data['Sector'], data['Industry'], data['MarketCapitalization'], data['EBITDA'], data['PERatio']]
        writer.writerow(info) 
        time.sleep(1)
        
    print("all data fetched")

# Filter and Sort the stock tickers

In [4]:
stock_df = pd.read_csv(f'{DATA_PATH}/tickers/stock_companies.csv')
stock_df = stock_df[stock_df['MarketCapitalization'] != "None"]

In [ ]:
stock_df['MarketCapitalization'] = pd.to_numeric(stock_df['MarketCapitalization'])

In [ ]:
stock_df = stock_df[stock_df['MarketCapitalization'] > 300 * 1000 * 1000]
print(f'There are {len(stock_df)} stocks')

In [5]:
sectors = stock_df['Sector'].unique()
print(f'There are {len(sectors)} unique sectors')
industries = stock_df['Industry'].unique()
print(f'There are {len(industries)} unique industries')

There are 7 unique sectors
There are 396 unique industries


In [6]:
print(sectors)

['LIFE SCIENCES' 'MANUFACTURING' 'REAL ESTATE & CONSTRUCTION'
 'TRADE & SERVICES' 'ENERGY & TRANSPORTATION' 'FINANCE' 'TECHNOLOGY']


In [ ]:
stock_df.to_csv(f'{DATA_PATH}/tickers/filtered_stock_companies.csv', index = False)  

# Russell 3000 Filter

In [ ]:
all_df = pd.read_csv(f'{DATA_PATH}/tickers/stock_companies.csv')
ru3000_df = pd.read_csv(f'{DATA_PATH}/tickers/ru3000.csv')
ru3000_df = ru3000_df.drop(['Company'], axis = 1)
joined_df = pd.merge(all_df, ru3000_df)

In [ ]:
filtered_tickers = pd.read_csv(f'{DATA_PATH}/tickers/filtered_stock_companies.csv')
filtered_tickers = filtered_tickers.drop(['Name','CIK','Sector','Industry','MarketCapitalization','EBITDA','PERatio'],axis=1)
joined_df = pd.merge(filtered_tickers, joined_df)

In [ ]:
joined_df = joined_df.drop_duplicates(subset=['Symbol'])
# joined_df = joined_df.drop(columns=['PERatio'])
joined_df.to_csv(f'{DATA_PATH}/tickers/filtered_ru3000.csv', index = False)  

In [ ]:
joined_df

# Fetching Daily Adjusted Time Series Data

In [ ]:
folder_path = f'{DATA_PATH}/daily'

def fetch_daily_data_to_csv(symbol):
    path = f'{folder_path}/{symbol}.csv'
    
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&datatype=csv&outputsize=full&apikey={API_KEY}'
    r = requests.get(url)

    if r.status_code != 200:
        print(f"Incorrect status code for symbol {symbol}. Expected 200, received {r.status_code}")
        return False

    data = r.text

    if not 'timestamp' in data:
        print(f"Problem with symbol {symbol}. Received \n{data}")
        return False

    cr = csv.reader(data.splitlines(), delimiter=',')

    with open(path, 'w+', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(cr)

    return True


filtered_df = pd.read_csv(f'{DATA_PATH}/tickers/filtered_ru3000.csv')
filtered_df = filtered_df.sort_values(by=['MarketCapitalization'], ascending=False)

i = 0
for (id, row) in filtered_df.iterrows():
    symbol = row[0]
    path = f'{folder_path}/{symbol}.csv'
    
    if os.path.exists(path):
        i = i+1
        print(f'{i}/{len(filtered_df)} symbol {symbol} already exists')
        continue
    
    successful = fetch_daily_data_to_csv(symbol);
    if (successful):
        i = i+1
        print(f'{i}/{len(filtered_df)} symbol {symbol} fetch complete')
    else:
        time.sleep(6)
        successful = fetch_daily_data_to_csv(symbol);
        if (successful):
            i = i+1
            print(f'{i}/{len(filtered_df)} symbol {symbol} fetch complete')

print(f"{i}/{len(filtered_df)} data fetched")


# Fetching Earnings Data

In [ ]:
folder_path = f'{DATA_PATH}/earnings_report'

def fetch_earnings_data_to_csv(symbol):
    path = f'{folder_path}/{symbol}.csv'
    
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={API_KEY}'
    r = requests.get(url)

    if r.status_code != 200:
        print(f"Incorrect status code for symbol {symbol}. Expected 200, received {r.status_code}")
        return False

    data = r.json()

    if not 'quarterlyEarnings' in data:
        print(f"Problem with symbol {symbol}. Received \n{data}")
        return False

    data = data['quarterlyEarnings']

    field_names = ['fiscalDateEnding', 'reportedDate', 'reportedEPS', 'estimatedEPS', 'surprise', 'surprisePercentage']

    with open(path, 'w') as f:
        writer = csv.DictWriter(f, fieldnames=field_names)
        writer.writeheader()
        writer.writerows(data)

    return True


filtered_df = pd.read_csv(f'{DATA_PATH}/tickers/filtered_ru3000.csv')

i = 0
for (id, row) in filtered_df.iterrows():
    symbol = row[0]
    path = f'{folder_path}/{symbol}.csv'

    if os.path.exists(path):
        i = i+1
        print(f'{i}/{len(filtered_df)} symbol {symbol} already exists')
        continue

    successful = fetch_earnings_data_to_csv(symbol)
    if (successful):
        i = i+1
        print(f'{i}/{len(filtered_df)} symbol {symbol} fetch complete')
    else:
        time.sleep(6)
        successful = fetch_earnings_data_to_csv(symbol)
        if (successful):
            i = i+1
            print(f'{i}/{len(filtered_df)} symbol {symbol} fetch complete')

print(f"{i}/{len(filtered_df)} data fetched")


# Fetching Intraday Data

In [ ]:
interval = '5min'
folder_path = f'{DATA_PATH}/intraday_{interval}'

filtered_df = pd.read_csv(f'{DATA_PATH}/tickers/filtered_ru3000.csv')
filtered_df = filtered_df.sort_values(by=['MarketCapitalization'], ascending=False)

def try_fetch(url, symbol):
    r = requests.get(url)
    
    if r.status_code != 200:
        print(f"Incorrect status code for symbol {symbol}. Expected 200, received {r.status_code}")
        return (False, False)

    def process_request(r):
        try:
            decoded_content = r.content.decode('utf-8')
            raw_data = StringIO(decoded_content)
            return pd.read_csv(raw_data)
        except:
            return pd.DataFrame()
    
    r = requests.get(url)
    df = process_request(r)
    if not 'time' in df:
        print("sleep for 6 seconds due to API limit")
        time.sleep(6)
        
        r = requests.get(url)
        df = process_request(r)
        if not 'time' in df:
            print(f"Problem with symbol {symbol}. Received \n{data}")
            return (False, False)
    return (True, r)

i = 0
incomplete_stocks = []
for (id, row) in filtered_df.iterrows():
    symbol = row[0]
    path = f'{folder_path}/{symbol}.csv'

    frames = []
    
    if os.path.exists(path):
        i = i+1
        print(f'{i}/{len(filtered_df)} symbol {symbol} already exists')
        continue
        
    incomplete_data = False

    with requests.Session() as s:
        for y in range(1,3):
            if (incomplete_data):
                break
            for m in range(1,13):
                url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol={symbol}&interval={interval}&slice=year{y}month{m}&apikey={API_KEY}'
                (success, download) = try_fetch(url, symbol)
                if success:
                    decoded_content = download.content.decode('utf-8')
                    raw_data = StringIO(decoded_content)
                    df = pd.read_csv(raw_data)
                    if (len(df) < 200):
                        incomplete_data = True

                    frames.append(df)
                else:
                    incomplete_data = True

                if (incomplete_data):
                    break
            if (incomplete_data):
                break

        if (incomplete_data):
            incomplete_stocks.append(symbol)
            print(f'{symbol} is problematic')
            continue

    # dataframe that combines 12 months worth of data
    df = pd.concat(frames)
    # round off the decimal places
    df['open'] = df['open'].astype(float).round(3)
    df['high'] = df['high'].astype(float).round(3)
    df['low'] = df['low'].astype(float).round(3)
    df['close'] = df['close'].astype(float).round(3)
    
    df.to_csv(path, sep=',', encoding='utf-8', index=False)

    i = i+1
    print(f'{i}/{len(filtered_df)} symbol {symbol} fetch complete')
    
print(f"{i}/{len(filtered_df)} data fetched")

# Temp

In [ ]:
folder_path = f'{DATA_PATH}/daily'

def fetch_daily_data_to_csv(symbol):
    path = f'{folder_path}/{symbol}.csv'
    
    if os.path.exists(path):
        print(f'symbol {symbol} already exists')
        return
        
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&datatype=csv&outputsize=full&apikey={API_KEY}'
    r = requests.get(url)

    if r.status_code != 200:
        print(f"Incorrect status code for symbol {symbol}. Expected 200, received {r.status_code}")
        return

    data = r.text

    if not 'timestamp' in data:
        print(f"Problem with symbol {symbol}. Received \n{data}")
        return

    cr = csv.reader(data.splitlines(), delimiter=',')

    with open(path, 'w+', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(cr)

    return True

fetch_daily_data_to_csv('QQQ')
fetch_daily_data_to_csv('SPY')
fetch_daily_data_to_csv('IWM')
fetch_daily_data_to_csv('TLT')
fetch_daily_data_to_csv('GOVT')
fetch_daily_data_to_csv('VTI')
fetch_daily_data_to_csv('VXX')
fetch_daily_data_to_csv('VXZ')
fetch_daily_data_to_csv('VIXY')
fetch_daily_data_to_csv('VIXM')